"""
ALTER TABLE general.oscar_awards_df_data
    RENAME COLUMN name TO nominee;

ALTER TABLE general.tmdb_df_data
    DROP COLUMN index;

DROP TABLE IF EXISTS movie_data;

CREATE TABLE movie_data AS
WITH
	cte_cast_crew AS (
		SELECT
			crewdd."TMDb_ID",
			ARRAY_AGG(crewdd.original_name) AS Movie_Cast,
			ARRAY_AGG(castdd.original_name) AS Movie_Crew
		FROM general.crew_df_data AS crewdd
		JOIN general.cast_df_data AS castdd
		 ON crewdd."TMDb_ID" = castdd."TMDb_ID"
		GROUP BY crewdd."TMDb_ID"
),
    cte_soundtrack_credits AS (
        SELECT
            "imdb_movie_ID",
            ARRAY_AGG("song_title") AS Soundtrack_Songs,
            ARRAY_AGG(DISTINCT "artist_name") AS Soundtrack_Artists
        FROM general.soundtrack_df_data
        GROUP BY "imdb_movie_ID"
),
	cte_merged_data AS (
		SELECT DISTINCT
			odd.*,
			tdd."US_release_date",
			tdd."GB_release_date",
			tdd."Alternative_Titles",
			tdd."Keyword_List",
			tdd."Budget",
			bdd2."Opening_Weekend_Gross",
			bdd2."Worldwide_Gross",
			ccc.Movie_Cast,
			ccc.Movie_Crew,
			stcdd.Soundtrack_Songs,
			stcdd.Soundtrack_Artists
 		FROM general.omdb_df_data AS odd
 		LEFT JOIN general.tmdb_df_data AS tdd
 		 ON tdd."IMDb_ID" = odd."imdbID"
 		LEFT JOIN cte_cast_crew AS ccc
 		 ON ccc."TMDb_ID" = tdd."TMDb_ID"
  		LEFT JOIN cte_soundtrack_credits AS stcdd
  		 ON stcdd."imdb_movie_ID" = odd."imdbID"
  		LEFT JOIN general.box_office_df_data AS bdd2
 		 ON odd."imdbID" = bdd2."IMDb_ID"
 )
(SELECT
	cterd.*,
	CARDINALITY(cterd."Alternative_Titles"::TEXT[]) AS Alternative_Titles
 FROM cte_merged_data AS cterd);

ALTER TABLE movie_data
    DROP COLUMN "Alternative_Titles",
    DROP COLUMN "Type",
    ALTER COLUMN "Runtime" SET DATA TYPE INT USING "Runtime"::INT,
    ALTER COLUMN "Genre" SET DATA TYPE TEXT[] USING "Genre"::TEXT[],
    ALTER COLUMN "Writer" SET DATA TYPE TEXT[] USING "Writer"::TEXT[],
    ALTER COLUMN "Actors" SET DATA TYPE TEXT[] USING "Actors"::TEXT[],
    ALTER COLUMN "Production" SET DATA TYPE TEXT[] USING "Production"::TEXT[],
    ALTER COLUMN "Language" SET DATA TYPE TEXT[] USING "Language"::TEXT[],
    ALTER COLUMN "Country" SET DATA TYPE TEXT[] USING "Country"::TEXT[],
    ALTER COLUMN "imdbVotes" SET DATA TYPE INT USING "imdbVotes"::INT,
    ALTER COLUMN "imdbRating" SET DATA TYPE NUMERIC USING "imdbRating"::NUMERIC,
    ALTER COLUMN "Keyword_List" SET DATA TYPE TEXT[] USING "Keyword_List"::TEXT[],
    ALTER COLUMN "Opening_Weekend_Gross" SET DATA TYPE NUMERIC USING "Opening_Weekend_Gross"::NUMERIC,
    ALTER COLUMN "Worldwide_Gross" SET DATA TYPE NUMERIC USING "Worldwide_Gross"::NUMERIC;

ALTER TABLE general.actor_data_df
    DROP COLUMN index;

ALTER TABLE general.name_id_df
    DROP COLUMN index;

ALTER TABLE general.grammy_awards_df_data
    DROP COLUMN index;

ALTER TABLE general.oscar_awards_df_data
    DROP COLUMN index;

ALTER TABLE general.gg_awards_df_data
    DROP COLUMN index;

DROP TABLE IF EXISTS actor_data;

CREATE TABLE actor_data AS
WITH cte_actor_data AS (
	SELECT
	    add2.*,
		nidd."names" AS Actor
	FROM general.actor_data_df AS add2
	JOIN general.name_id_df AS nidd
	 ON add2."IMDb_ID" = nidd.imdb_ids
)
(SELECT DISTINCT * FROM cte_actor_data);


SELECT md.*, tdd.*
 FROM movie_data md
 LEFT JOIN general.tmdb_df_data tdd
  ON md."imdbID" = tdd."IMDb_ID"
 WHERE
 	md."Released" IS NULL;


ALTER TABLE general.gg_awards_df_data
    RENAME COLUMN year_film TO "film_year";

ALTER TABLE general.gg_awards_df_data
    RENAME COLUMN year_award TO "awards_year";

ALTER TABLE general.gg_awards_df_data
    RENAME COLUMN win TO "winner";

ALTER TABLE general.oscar_awards_df_data
    RENAME COLUMN year_film TO "film_year";

ALTER TABLE general.oscar_awards_df_data
    RENAME COLUMN year_ceremony TO "ceremony_year";
"""